In [ ]:
%matplotlib inline
import os
import sys
import math
import datetime
import numpy as np
from numpy.random import random
import pandas as pd
import pyworld
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from glob import glob
from IPython.display import Audio
from tensorflow.keras import *
from tensorflow.keras.utils import *
from tensorflow.keras.layers import *
from tensorflow.keras.activations import sigmoid, relu, tanh

In [ ]:
tf.random.set_seed(115)

In [ ]:
class DataLoader():
    def __init__(self, dataset_a, dataset_b, sampling_rate = 22050, num_mcep = 32, frame_period = 5.0, n_frames = 128):
        self.sampling_rate = sampling_rate
        self.num_mcep = num_mcep
        self.frame_period = frame_period
        self.n_frames = n_frames
        self.dataset_a = dataset_a
        self.dataset_b = dataset_b
       
    def test(self, wav_path_A, wav_path_B, net_AB, net_BA):
        wavs_A = self.load_wavs(wav_dir = wav_path_A, sr = self.sampling_rate)
        wavs_B = self.load_wavs(wav_dir = wav_path_B, sr = self.sampling_rate)
        f0s_A, timeaxes_A, sps_A, aps_A, coded_sps_A = self.world_encode_data(wavs = wavs_A, fs = self.sampling_rate, frame_period = self.frame_period, coded_dim = self.num_mcep)
        f0s_B, timeaxes_B, sps_B, aps_B, coded_sps_B = self.world_encode_data(wavs = wavs_B, fs = self.sampling_rate, frame_period = self.frame_period, coded_dim = self.num_mcep)
        log_f0s_mean_A, log_f0s_std_A = self.logf0_statistics(f0s_A)
        log_f0s_mean_B, log_f0s_std_B = self.logf0_statistics(f0s_B)
        coded_sps_A_transposed = self.transpose_in_list(lst = coded_sps_A)
        coded_sps_B_transposed = self.transpose_in_list(lst = coded_sps_B)      
        
        coded_sps_A_norm, coded_sps_A_mean, coded_sps_A_std = self.coded_sps_normalization_fit_transoform(coded_sps = coded_sps_A_transposed)
        coded_sps_B_norm, coded_sps_B_mean, coded_sps_B_std = self.coded_sps_normalization_fit_transoform(coded_sps = coded_sps_B_transposed) 
                
        wav_transformed_A = self.transfer(wav_path_A, net_AB, coded_sps_A_mean, coded_sps_A_std, coded_sps_B_mean, coded_sps_B_std, log_f0s_mean_A, log_f0s_std_A, log_f0s_mean_B, log_f0s_std_B)
        wav_transformed_B = self.transfer(wav_path_B, net_BA, coded_sps_B_mean, coded_sps_B_std, coded_sps_A_mean, coded_sps_A_std, log_f0s_mean_B, log_f0s_std_B, log_f0s_mean_A, log_f0s_std_A)
        
        return (wav_transformed_A, wav_transformed_B)

    def transfer(self, filepath, net, coded_sps_A_mean, coded_sps_A_std, coded_sps_B_mean, coded_sps_B_std, log_f0s_mean_A, log_f0s_std_A, log_f0s_mean_B, log_f0s_std_B):
        sampling_rate = self.sampling_rate
        frame_period = self.frame_period
        num_mcep = self.num_mcep
        n_frames = self.n_frames
        wav, _ = librosa.load(filepath, sr = sampling_rate, mono = True)
        wav = self.wav_padding(wav = wav, sr = sampling_rate, frame_period = frame_period, multiple = 4)
        f0, timeaxis, sp, ap = self.world_decompose(wav = wav, fs = sampling_rate, frame_period = frame_period)
        f0_converted = self.pitch_conversion(f0 = f0, mean_log_src = log_f0s_mean_A, std_log_src = log_f0s_std_A, mean_log_target = log_f0s_mean_B, std_log_target = log_f0s_std_B)
        coded_sp = self.world_encode_spectral_envelop(sp = sp, fs = sampling_rate, dim = num_mcep)
        coded_sp_transposed = coded_sp.T
        coded_sp_norm = (coded_sp_transposed - coded_sps_A_mean) / coded_sps_A_std
        coded_sp_norm = self.sample_test_data(coded_sp_norm, n_frames = n_frames)     
        coded_sp_norm = tf.expand_dims(coded_sp_norm, -1)    
        coded_sp_converted_norm = net.predict(coded_sp_norm)
        coded_sp_converted_norm = self.join_test_data(coded_sp_converted_norm, f0, n_frames) 
        coded_sp_converted = coded_sp_converted_norm * coded_sps_B_std + coded_sps_B_mean
        coded_sp_converted = coded_sp_converted.T
        coded_sp_converted = np.ascontiguousarray(coded_sp_converted)
        decoded_sp_converted = self.world_decode_spectral_envelop(coded_sp = coded_sp_converted, fs = sampling_rate)
        wav_transformed = self.world_speech_synthesis(f0 = f0_converted, decoded_sp = decoded_sp_converted, ap = ap, fs = sampling_rate, frame_period = frame_period)
        return wav_transformed
    
    def load_wavs(self, wav_dir, sr):

        wavs = list()
        wav, _ = librosa.load(wav_dir, sr = sr, mono = True)
        #wav = wav.astype(np.float64)
        wavs.append(wav)

        return wavs

    def world_decompose(self, wav, fs, frame_period = 5.0):

        # Decompose speech signal into f0, spectral envelope and aperiodicity using WORLD
        wav = wav.astype(np.float64)
        f0, timeaxis = pyworld.harvest(wav, fs, frame_period = frame_period, f0_floor = 71.0, f0_ceil = 800.0)
        sp = pyworld.cheaptrick(wav, f0, timeaxis, fs)
        ap = pyworld.d4c(wav, f0, timeaxis, fs)

        return f0, timeaxis, sp, ap

    def world_encode_spectral_envelop(self, sp, fs, dim = 24):

        # Get Mel-cepstral coefficients (MCEPs)

        #sp = sp.astype(np.float64)
        coded_sp = pyworld.code_spectral_envelope(sp, fs, dim)

        return coded_sp

    def world_decode_spectral_envelop(self, coded_sp, fs):

        fftlen = pyworld.get_cheaptrick_fft_size(fs)
        #coded_sp = coded_sp.astype(np.float32)
        #coded_sp = np.ascontiguousarray(coded_sp)
        decoded_sp = pyworld.decode_spectral_envelope(coded_sp, fs, fftlen)

        return decoded_sp


    def world_encode_data(self, wavs, fs, frame_period = 5.0, coded_dim = 24):

        f0s = list()
        timeaxes = list()
        sps = list()
        aps = list()
        coded_sps = list()

        for wav in wavs:
            f0, timeaxis, sp, ap = self.world_decompose(wav = wav, fs = fs, frame_period = frame_period)
            coded_sp = self.world_encode_spectral_envelop(sp = sp, fs = fs, dim = coded_dim)
            f0s.append(f0)
            timeaxes.append(timeaxis)
            sps.append(sp)
            aps.append(ap)
            coded_sps.append(coded_sp)

        return f0s, timeaxes, sps, aps, coded_sps


    def transpose_in_list(self, lst):

        transposed_lst = list()
        for array in lst:
            transposed_lst.append(array.T)
        return transposed_lst


    def world_decode_data(self, coded_sps, fs):

        decoded_sps =  list()

        for coded_sp in coded_sps:
            decoded_sp = self.world_decode_spectral_envelop(coded_sp, fs)
            decoded_sps.append(decoded_sp)

        return decoded_sps


    def world_speech_synthesis(self, f0, decoded_sp, ap, fs, frame_period):

        #decoded_sp = decoded_sp.astype(np.float64)
        wav = pyworld.synthesize(f0, decoded_sp, ap, fs, frame_period)
        # Librosa could not save wav if not doing so
        wav = wav.astype(np.float32)

        return wav


    def world_synthesis_data(self, f0s, decoded_sps, aps, fs, frame_period):

        wavs = list()

        for f0, decoded_sp, ap in zip(f0s, decoded_sps, aps):
            wav = self.world_speech_synthesis(f0, decoded_sp, ap, fs, frame_period)
            wavs.append(wav)

        return wavs


    def coded_sps_normalization_fit_transoform(self, coded_sps):

        coded_sps_concatenated = np.concatenate(coded_sps, axis = 1)
        coded_sps_mean = np.mean(coded_sps_concatenated, axis = 1, keepdims = True)
        coded_sps_std = np.std(coded_sps_concatenated, axis = 1, keepdims = True)

        coded_sps_normalized = list()
        for coded_sp in coded_sps:
            coded_sps_normalized.append((coded_sp - coded_sps_mean) / coded_sps_std)
        
        return coded_sps_normalized, coded_sps_mean, coded_sps_std

    def coded_sps_normalization_transoform(coded_sps, coded_sps_mean, coded_sps_std):

        coded_sps_normalized = list()
        for coded_sp in coded_sps:
            coded_sps_normalized.append((coded_sp - coded_sps_mean) / coded_sps_std)
        
        return coded_sps_normalized

    def coded_sps_normalization_inverse_transoform(self, normalized_coded_sps, coded_sps_mean, coded_sps_std):

        coded_sps = list()
        for normalized_coded_sp in normalized_coded_sps:
            coded_sps.append(normalized_coded_sp * coded_sps_std + coded_sps_mean)

        return coded_sps

    def coded_sp_padding(self, coded_sp, multiple = 4):

        num_features = coded_sp.shape[0]
        num_frames = coded_sp.shape[1]
        num_frames_padded = int(np.ceil(num_frames / multiple)) * multiple
        num_frames_diff = num_frames_padded - num_frames
        num_pad_left = num_frames_diff // 2
        num_pad_right = num_frames_diff - num_pad_left
        coded_sp_padded = np.pad(coded_sp, ((0, 0), (num_pad_left, num_pad_right)), 'constant', constant_values = 0)

        return coded_sp_padded

    def wav_padding(self, wav, sr, frame_period, multiple = 4):

        assert wav.ndim == 1 
        num_frames = len(wav)
        num_frames_padded = int((np.ceil((np.floor(num_frames / (sr * frame_period / 1000)) + 1) / multiple + 1) * multiple - 1) * (sr * frame_period / 1000))
        num_frames_diff = num_frames_padded - num_frames
        num_pad_left = num_frames_diff // 2
        num_pad_right = num_frames_diff - num_pad_left
        wav_padded = np.pad(wav, (num_pad_left, num_pad_right), 'constant', constant_values = 0)

        return wav_padded


    def logf0_statistics(self, f0s):

        log_f0s_concatenated = np.ma.log(np.concatenate(f0s))
        log_f0s_mean = log_f0s_concatenated.mean()
        log_f0s_std = log_f0s_concatenated.std()

        return log_f0s_mean, log_f0s_std

    def pitch_conversion(self, f0, mean_log_src, std_log_src, mean_log_target, std_log_target):

        # Logarithm Gaussian normalization for Pitch Conversions
        f0_converted = np.exp((np.log(f0) - mean_log_src) / std_log_src * std_log_target + mean_log_target)

        return f0_converted

    def wavs_to_specs(self, wavs, n_fft = 1024, hop_length = None):

        stfts = list()
        for wav in wavs:
            stft = librosa.stft(wav, n_fft = n_fft, hop_length = hop_length)
            stfts.append(stft)

        return stfts


    def wavs_to_mfccs(self, wavs, sr, n_fft = 1024, hop_length = None, n_mels = 128, n_mfcc = 24):

        mfccs = list()
        for wav in wavs:
            mfcc = librosa.feature.mfcc(y = wav, sr = sr, n_fft = n_fft, hop_length = hop_length, n_mels = n_mels, n_mfcc = n_mfcc)
            mfccs.append(mfcc)

        return mfccs


    def mfccs_normalization(self, mfccs):

        mfccs_concatenated = np.concatenate(mfccs, axis = 1)
        mfccs_mean = np.mean(mfccs_concatenated, axis = 1, keepdims = True)
        mfccs_std = np.std(mfccs_concatenated, axis = 1, keepdims = True)

        mfccs_normalized = list()
        for mfcc in mfccs:
            mfccs_normalized.append((mfcc - mfccs_mean) / mfccs_std)
        
        return mfccs_normalized, mfccs_mean, mfccs_std


    def sample_train_data(self, dataset_A, dataset_B, n_frames = 128):

        num_samples = min(len(dataset_A), len(dataset_B))
        train_data_A_idx = np.arange(len(dataset_A))
        train_data_B_idx = np.arange(len(dataset_B))
        np.random.shuffle(train_data_A_idx)
        np.random.shuffle(train_data_B_idx)
        train_data_A_idx_subset = train_data_A_idx[:num_samples]
        train_data_B_idx_subset = train_data_B_idx[:num_samples]

        train_data_A = list()
        train_data_B = list()

        for idx_A, idx_B in zip(train_data_A_idx_subset, train_data_B_idx_subset):
            data_A = dataset_A[idx_A]
            frames_A_total = data_A.shape[1]
            assert frames_A_total >= n_frames
            start_A = np.random.randint(frames_A_total - n_frames + 1)
            end_A = start_A + n_frames
            train_data_A.append(data_A[:,start_A:end_A])

            data_B = dataset_B[idx_B]
            frames_B_total = data_B.shape[1]
            assert frames_B_total >= n_frames
            start_B = np.random.randint(frames_B_total - n_frames + 1)
            end_B = start_B + n_frames
            train_data_B.append(data_B[:,start_B:end_B])

        train_data_A = np.array(train_data_A)
        train_data_B = np.array(train_data_B)

        return train_data_A, train_data_B
    
    def sample_test_data(self, data, n_frames = 128):
        num_samples = data.shape[1]//n_frames
        test_data = list()
        for idx in range(num_samples):
            start = idx*n_frames
            end = start + n_frames
            test_data.append(data[:,start:end])
        test_data.append(data[:,-n_frames:])
        test_data = np.array(test_data)
        return test_data
    def join_test_data(self, data, f0, n_frames = 128):
        overlap = n_frames-(len(f0)%n_frames)
        num_samples = data.shape[0]
        joined_data = data[0]
        for idx in range(num_samples-2):
            joined_data = tf.concat([joined_data, data[idx+1]], 1)
        joined_data = tf.concat([joined_data, data[-1,:,overlap:,:]], 1)    
        joined_data = tf.squeeze(joined_data)
        joined_data = np.array(joined_data)
        return joined_data

In [ ]:
# Build the discriminators
model_path = 'C:/Python程式存放處/'
d_A = models.load_model(model_path + 'U_OSL_d_A.h5')
d_B = models.load_model(model_path + 'U_OSL_d_B.h5')
        
# Build the generators
g_AB = models.load_model(model_path + 'U_OSL_g_AB.h5')
g_BA = models.load_model(model_path + 'U_OSL_g_BA.h5')

In [ ]:
a = 'C:/Python程式存放處/vcc2018_training/VCC2SF2/10001.wav' #女生轉換前
b = 'C:/Python程式存放處/vcc2018_training/VCC2SM1/10001.wav' #男生轉換前

In [ ]:
testloader = DataLoader(dataset_a="n",dataset_b="r")
(a_fake, b_fake) = testloader.test(wav_path_A=a,wav_path_B=b,net_AB=net.g_AB,net_BA=net.g_BA)

In [ ]:
librosa.output.write_wav('C:/Python程式存放處/vcc2018_wav_after_transfer/fake_M.wav', a_fake, 22050)
librosa.output.write_wav('C:/Python程式存放處/vcc2018_wav_after_transfer/fake_F.wav', b_fake, 22050)